<a href="https://colab.research.google.com/github/erberry/ThinkML/blob/main/%E8%BF%90%E8%A1%8C%E5%A4%A7%E8%AF%AD%E8%A8%80%E6%A8%A1%E5%9E%8B_chatglm2_6b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 在 GPU 上推理

#### 查看当前 gpu 信息

colab 的 T4 GPU可以免费使用，显存 16GB

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jul  5 06:11:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#### 下载 ChatGLM2-6B 代码，并安装依赖

In [ ]:
!git clone https://github.com/THUDM/ChatGLM2-6B.git

Cloning into 'ChatGLM2-6B'...
remote: Enumerating objects: 175, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 175 (delta 64), reused 62 (delta 48), pack-reused 81
Receiving objects: 100% (175/175), 5.09 MiB | 33.02 MiB/s, done.
Resolving deltas: 100% (88/88), done.


In [ ]:
!cd ChatGLM2-6B && pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.6/416.6 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.0 MB/s eta 0:00:

#### 推理

首次运行需要下载模型文件（大约13GB），耐心等待

如果显存有限，可以修改 cli_demo.py 文件，使用量化模型进行推理：

将

```
model = AutoModel.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True).cuda()
```
修改为

```
model = AutoModel.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True).quantize(8).cuda()
```
或者

```
model = AutoModel.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True).quantize(4).cuda()
```

量化模型将模型参数精度降低，以减少模型对内存的占用，但也会会降低推理准确度。

如果没有显卡，可以使用 chatglm.cpp 在CPU上推理，但是推理速度会比使用GPU慢很多。 在CPU上推理见下方 【使用 chatglm.cpp 在CPU上推理】



In [ ]:
!cd ChatGLM2-6B && python cli_demo.py

2023-07-05 06:23:16.192149: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading checkpoint shards: 100% 7/7 [01:08<00:00,  9.84s/it]
欢迎使用 ChatGLM2-6B 模型，输入内容即可进行对话，clear 清空对话历史，stop 终止程序

用户：描述蓝色

ChatGLM：蓝色是一种基本颜色,通常被认为是一种平静、冷静和信任感和平静的颜色。蓝色也代表着许多不同的情感,例如天空、海洋、湖泊、蓝色代表信任、稳定、成熟和宁静等。在不同的文化和传统中,蓝色也具有不同的象征意义。

用户：翻译这句话 “你好，请问天安门怎么走？“ 到英文

ChatGLM："你好,请问天安门怎么走?" 的英文翻译是:"Hello, how do you go to Tiananmen Square?"

用户：写一首关于爱得诗  

ChatGLM：爱得诗

爱,是生命中最美好的情感
它能让我们心灵颤抖
让我们感到幸福和充实
它能让我们看到世界的美

爱,像春风拂面一样温暖
它能抚摸我们的灵魂
让我们感受到生命的意义
它能让我们看到未来的光明

爱,像一片湖水一样平静
它能让我们心灵宁静
让我们感受到内心的力量
它能让我们看到生命的真谛

爱,像一朵鲜花一样美丽
它能让我们感受到生命的美好
它能让我们感受到生命的温暖
它能让我们感受到生命的意义

爱,是生命中最美好的情感
它能让我们心灵颤抖
让我们感到幸福和充实
它能让我们看到世界的美

用户：解释重力得概念

ChatGLM：重力得概念是指物体由于地球引力的作用而受到的力。根据牛顿第二定律,物体的加速度与作用在物体上的力成正比,与物体的质量成反比。因此,当一个物体在地球上时,它受到的重力作用于它的质量上,导致它产生加速度。重力的方向总是竖直向下的,因此物体会沿着重力方向上产生加速度。

用户：蒙娜丽莎是谁画的

ChatGLM：蒙娜丽莎(Mona Lisa)是由意大利艺术家达·芬奇(Leonardo da Vinci)于16世纪早期绘制的一幅肖像画。达·芬奇是文艺复兴时期最著名的艺术家之一,他的绘画风格和

### 使用 chatglm.cpp 在CPU上推理



#### 下载 ChatGLM2-6B 和 chatglm.cpp 代码，并安装依赖的 python 包。

In [ ]:
!git clone https://github.com/THUDM/ChatGLM2-6B.git
!cd ChatGLM2-6B && pip install -r requirements.txt

In [ ]:
!git clone --recursive https://github.com/li-plus/chatglm.cpp.git


Cloning into 'chatglm.cpp'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 125 (delta 22), reused 19 (delta 13), pack-reused 89
Receiving objects: 100% (125/125), 783.87 KiB | 13.06 MiB/s, done.
Resolving deltas: 100% (60/60), done.


#### 对原模型进行量化。可选的量化类型如下：

- q4_0: 4-bit integer quantization with fp16 scales.
- q4_1: 4-bit integer quantization with fp16 scales and minimum values.
- q5_0: 5-bit integer quantization with fp16 scales.
- q5_1: 5-bit integer quantization with fp16 scales and minimum values.
- q8_0: 8-bit integer quantization with fp16 scales.
- f16: half precision floating point weights without quantization.
- f32: single precision floating point weights without quantization.

以上类型代表了模型参数的精度，精度越高，模型推理所需要的内存空间越大，推理越准确。反之降低精度可以降低对内存大小的要求，但也会影响推理的准确性。

这里选择 f16。大约需要13GB内存。

由于需要下载原模型（大约16GB），执行比较慢。耐心等待！

In [ ]:
!cd chatglm.cpp && python3 convert.py -i THUDM/chatglm2-6b -t f16 -o chatglm2-ggml-f16.bin

A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm2-6b:
- tokenization_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm2-6b:
- configuration_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm2-6b:
- quantization.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm2-6b:
- modeling_chatglm.py
- quantization.py
. Make sure to double-check they do not con

#### 编译 chatglm.cpp

In [ ]:
!cd chatglm.cpp && cmake -B build && cmake --build build -j

-- The C compiler identification is GNU 9.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Failed
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threads: TRUE  
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- x86 detected
-- Linux detected
-- VERSION: 0.2.00
-- Not Found TCMalloc: TCMALLOC_LIB-NOTFOUND
-- Configuring done
-- Generating done
-- Build files have been written to: /content/chatglm.cpp/build
[  1%] Building C object third_party/ggml/src/CMakeFiles/ggml.dir/ggml.c.o
[  2%] Building CXX object third_party/sentencepiece/src/CMakeFiles/sentencepiece_train-static.dir/builder.cc.o
[  4%] Building CXX object 

#### 开始推理。

-i 参数开启交互模式，其他参数 -h 查看

In [ ]:
!cd chatglm.cpp && ./build/bin/main -m chatglm2-ggml-f16.bin -i

    ________          __  ________    __  ___                 
   / ____/ /_  ____ _/ /_/ ____/ /   /  |/  /_________  ____  
  / /   / __ \/ __ `/ __/ / __/ /   / /|_/ // ___/ __ \/ __ \ 
 / /___/ / / / /_/ / /_/ /_/ / /___/ /  / // /__/ /_/ / /_/ / 
 \____/_/ /_/\__,_/\__/\____/_____/_/  /_(_)___/ .___/ .___/  
                                              /_/   /_/       
Prompt   > 你好
ChatGLM2 > 你好👋！我是人工智能助手 ChatGLM2-6B，很高兴见到你，欢迎问我任何问题。
Prompt   > 你能唱一首《我爱你中国》吗
ChatGLM2 > 抱歉，作为人工智能助手，我不具有唱歌的功能。但我可以为您提供《我爱你中国》歌词。

《我爱你中国》

作词：乔羽
作曲：刘炽

一千里送鹅毛，
礼送黄河岸。

九百九十九道弯，
留下丹心一片。

胸怀 type-script }

②怪工作效率低，
使人累得透不过气。
Prompt   > 详述一下光合作用的过程
ChatGLM2 > 光合作用是植物和某些微生物利用光能将二氧化碳和水合成有机物，同时释放氧气的过程。它发生在植物细胞的叶绿体中，是一种重要的生命活动。光合作用的过程可以分为两个阶段：光反应和暗反应。

光反应阶段在光照下进行，主要负责吸收光能并将其转化为化学能。该过程包括两个步骤：光系统 I 和光系统 II。

1. 光系统 II：又称叶绿素光系统，负责吸收光能。它包括一个特殊的叶绿素分子，称为叶绿素 a，还有一个辅助色素——叶绿素 b。当光子照射到叶绿素 a 时，叶绿素 a 的一个电子被光子激发，形成一个高能电子。这个高能电子随后撞
Prompt   > 生命的意义是什么？
ChatGLM2 > 生命的意义是一个非常深刻和哲学性的问题，没办法用三言两语来回答。每个人对生命的意义都有自己的看法和定义。

从生物